<a href="https://colab.research.google.com/github/nzaw96/Predicting-Grain-Production-in-Ukraine/blob/main/Ukraine_hackathon_MLmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse # root mean squared error prefered


In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 2.8 MB/s 


In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
df_ukr = pd.read_csv('ukr_data.csv')

In [ ]:
df_ukr.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df_ukr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Commodity              140 non-null    object 
 1   Year                   140 non-null    int64  
 2   Country                140 non-null    object 
 3   Area Harvested         140 non-null    int64  
 4   Beginning Stocks       140 non-null    int64  
 5   Production             140 non-null    int64  
 6   Imports                140 non-null    int64  
 7   Total Supply           140 non-null    int64  
 8   Exports                140 non-null    int64  
 9   Feed Dom. Consumption  140 non-null    int64  
 10  Fsi Consumption        140 non-null    int64  
 11  Domestic Consumption   140 non-null    int64  
 12  Ending Stocks          140 non-null    int64  
 13  Yield                  140 non-null    float64
 14  Population             140 non-null    float64
 15  Rural 

In [ ]:
def adj_Rsquared(rsq, n, k): # takes in R-squared val (rsq) and return the adjusted R squared val (n: # samples, k: # independent vars)
  adj_rsq = 1 - ((1 - rsq)*(n-1)/(n-k-1))
  return adj_rsq

In [ ]:
df_ukr.columns

Index(['Commodity', 'Year', 'Country', 'Area Harvested', 'Beginning Stocks',
       'Production', 'Imports', 'Total Supply', 'Exports',
       'Feed Dom. Consumption', 'Fsi Consumption', 'Domestic Consumption',
       'Ending Stocks', 'Yield', 'Population', 'Rural Population'],
      dtype='object')

In [ ]:
# Now we look to drop the columns to further reduce the dimensionality of the data since each dataset has 140 samples each.

df_ukr.drop(columns=['Country'], inplace=True)

In [ ]:
# Feature aggregation: Domestic Consumption = Feed Dom. Consumption + Fsi Consumption. So can drop the latter two columns

df_ukr.drop(columns=['Feed Dom. Consumption', 'Fsi Consumption'], inplace=True)

In [ ]:
# Next we use the knowledge of the data to drop more columns.
# Total Supply = Production + Imports + Beginning Stocks.
# We don't need both total supply and production (0.99 correlation) in the data set. Can drop Total Supply.

df_ukr.drop(columns=['Total Supply'], inplace=True)

In [ ]:
df_ukr.head()

,Commodity,Year,Area Harvested,Beginning Stocks,Production,Imports,Exports,Domestic Consumption,Ending Stocks,Yield,Population,Rural Population
0,Barley,1987,4077,900,12190,230,1655,10817,848,2.99,51293000.0,34.170
1,Barley,1988,3658,848,8751,1160,1345,8578,836,2.39,51521000.0,33.601
2,Barley,1989,3234,836,10090,550,1090,9677,709,3.12,51773000.0,33.282
3,Barley,1990,2729,709,9168,500,350,9307,720,3.36,51891400.0,33.243
4,Barley,1991,3190,720,8047,435,275,8306,621,2.52,52000500.0,33.204


In [ ]:
# We will drop the year column as it won't contribute much to the ML models

df_ukr.drop(columns=['Year'], inplace=True)

In [ ]:
# (BIG CHANGE HERE) Dummy/one-hot encoding could increase the dimensionality, will try an alternative called Leave-one-out (target) encoding
from category_encoders import LeaveOneOutEncoder

encoder = LeaveOneOutEncoder(cols=['Commodity'], return_df=True)
X_ukr = df_ukr.drop(columns=['Production'])
y_ukr = df_ukr['Production']

X_enc_ukr = encoder.fit_transform(X_ukr, y_ukr)

X_enc_ukr.head()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Commodity,Area Harvested,Beginning Stocks,Imports,Exports,Domestic Consumption,Ending Stocks,Yield,Population,Rural Population
0,9094.441176,4077,900,230,1655,10817,848,2.99,51293000.0,34.170
1,9195.588235,3658,848,1160,1345,8578,836,2.39,51521000.0,33.601
2,9156.205882,3234,836,550,1090,9677,709,3.12,51773000.0,33.282
3,9183.323529,2729,709,500,350,9307,720,3.36,51891400.0,33.243
4,9216.294118,3190,720,435,275,8306,621,2.52,52000500.0,33.204


In [ ]:
# Since Beginning Stocks and Ending Stocks are just the same values shifted by one year, Don't need to include both.
# Will keep Beginning Stocks as it makes more sense.

X_enc_ukr.drop(columns=['Ending Stocks'], inplace=True)

In [ ]:
df_ukr.corr()['Production'].abs().sort_values(ascending=False)

Production              1.000000
Area Harvested          0.831971
Exports                 0.805837
Yield                   0.780656
Domestic Consumption    0.692079
Ending Stocks           0.660555
Beginning Stocks        0.515407
Rural Population        0.390395
Population              0.343611
Imports                 0.078349
Name: Production, dtype: float64

In [ ]:
X_enc_ukr.columns

Index(['Commodity', 'Area Harvested', 'Beginning Stocks', 'Imports', 'Exports',
       'Domestic Consumption', 'Yield', 'Population', 'Rural Population'],
      dtype='object')

In [ ]:
# Imports only have 0.078 correlation with Production for Ukraine so Can drop Imports.
X_enc_ukr.drop(columns=['Imports'], inplace=True)

In [ ]:
# Newest Edit: Dropping Population column and only keeping the percentage of rural population since the correlation betweeen the two is 0.95.

X_enc_ukr.drop(columns=['Population'], inplace=True)

In [ ]:

# Newest Edit: Doing the train_test split with 1000 different random_states and getting the average of 
# train and test errors for each model.

Xtrains = []
Xtests = []
Xtrains_pca = []
Xtests_pca = []
ytrains = []
ytests = []

pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_enc_ukr)

y = np.array(y_ukr) # changing from series to array

for i in range(1000):
  X_train, X_test, y_train, y_test = train_test_split(X_enc_ukr, y, test_size=0.2, random_state=i)
  X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=i)
  Xtrains.append(X_train)
  Xtests.append(X_test)
  Xtrains_pca.append(X_train_pca)
  Xtests_pca.append(X_test_pca)
  ytrains.append(y_train)
  ytests.append(y_test)




In [ ]:
# Writing a function that takes in a model and run it 1000 times for 1000 different train-test splits of the data.

def run_model(model, X_trains=Xtrains, X_tests=Xtests, y_trains=ytrains, y_tests=ytests, iter=1000, get_r2=False):
  train_errs, test_errs = [], []
  results = {} 
  if get_r2:
    train_r2_scores = []
    test_r2_scores = []
  for i in range(iter):
    model.fit(X_trains[i], y_trains[i])
    ypred_train = model.predict(X_trains[i])
    ypred_test = model.predict(X_tests[i])
    rmse_train = np.sqrt(mse(y_trains[i], ypred_train)) # Root-mean square Training Error
    rmse_test = np.sqrt(mse(y_tests[i], ypred_test)) # Root-mean square Testing Error
    train_errs.append(rmse_train)
    test_errs.append(rmse_test)
    if get_r2:
      train_r2 = r2_score(y_trains[i], ypred_train)
      train_adj_r2 = adj_Rsquared(train_r2, X_trains[i].shape[0], X_trains[i].shape[1])
      train_r2_scores.append(train_adj_r2)
      test_r2 = r2_score(y_tests[i], ypred_test)
      test_adj_r2 = adj_Rsquared(test_r2, X_tests[i].shape[0], X_tests[i].shape[1])
      test_r2_scores.append(test_adj_r2)
  results['Train Error'] = np.mean(train_errs)
  results['Test Error'] = np.mean(test_errs)
  if get_r2:
    results['Train R2 Score'] = np.mean(train_r2_scores)
    results['Test R2 Score'] = np.mean(test_r2_scores)
  return results



In [ ]:
# Linear Regression : baseline model

lm = LinearRegression()
res = run_model(lm, get_r2=True)
print(res)

{'Train Error': 948.4801917684855, 'Test Error': 1030.8388967658643, 'Train R2 Score': 0.9889972489826988, 'Test R2 Score': 0.9820616602091466}


In [ ]:
# Finding the optimal alpha value for ridge regression model through LOOCV
alpha_range = np.arange(1, 10, 0.01)
ridgeCV = RidgeCV(alphas=alpha_range) # cv parameter by default is LOOCV
ridgeCV.fit(X_enc_ukr, y) # Cross Validatio search of the alpha value is performed on the entire data set and NOT on a training set because there are 1000 different training sets to choose from
print(ridgeCV.alpha_) # 6.609999999999


6.610000000000005


In [ ]:
# Ridge Regression

ridge = Ridge(alpha=6.61) # setting alpha=1.0 for now, Will find alpha value again
res_ridge = run_model(ridge, get_r2=True)
print(res_ridge)

{'Train Error': 948.8882581122508, 'Test Error': 1030.0537275931033, 'Train R2 Score': 0.9889878418392044, 'Test R2 Score': 0.9820982663328809}


In [ ]:
# Finding the optimal alpha value for lasso regression model through LOOCV

loo = LeaveOneOut() # an instance of LOOCV
lassoCV = LassoCV(alphas=alpha_range, cv=loo) # Unlike Ridge LassoCV doesn't use LOOCV as the default CV

lassoCV.fit(X_enc_ukr, y)
print(lassoCV.alpha_) 

6.860000000000005


In [ ]:
# Lasso Regression

lasso = Ridge(alpha=6.86)
res_lasso = run_model(lasso, get_r2=True)
print(res_lasso)

{'Train Error': 948.9158161031321, 'Test Error': 1030.042314422415, 'Train R2 Score': 0.98898720642228, 'Test R2 Score': 0.9820989759210509}


In [ ]:
# Finding the optimal parameters for Decision Tree Regressor

# Now, let's try hyperparameter tuning
decTreeCV = DecisionTreeRegressor()
path = decTreeCV.cost_complexity_pruning_path(X_enc_ukr, y)
parameters={# "splitter":["best","random"],
            "max_depth" : list(range(3, 15)),
           #"min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
           #"min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           #"max_features":["auto","log2","sqrt",None],
           #"max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90],
            "ccp_alpha": path["ccp_alphas"]}

loo = LeaveOneOut()
gs = GridSearchCV(decTreeCV, param_grid=parameters, cv=loo, scoring='neg_mean_squared_error', n_jobs=-1) # searching for the best parameters
gs.fit(X_enc_ukr, y)

print(gs.best_score_)
print(gs.best_params_)

# print(gs.best_params_) # {'ccp_alpha': 1572.8036734686366, 'max_depth': 14}


-2780099.187515012
{'ccp_alpha': 1572.8036734686366, 'max_depth': 14}


In [ ]:
# Decision Tree Regression


decTree = DecisionTreeRegressor(ccp_alpha=0.8, max_depth=11) # default parameters for now
res_decTree = run_model(decTree)

print(res_decTree) # bench mark to improve upon: {'Train Error': 0.0, 'Test Error': 2345.6259226112484}

# after cv: {'Train Error': 8.523315607000674, 'Test Error': 2327.817099561585}

{'Train Error': 8.523315607000674, 'Test Error': 2327.817099561585}


In [ ]:
decTree.feature_importances_

array([7.17256854e-02, 7.18469152e-01, 1.43352209e-03, 1.00943860e-02,
       8.85819041e-03, 1.89095849e-01, 3.23214496e-04])

In [ ]:
# Random Forest Regression

rf = RandomForestRegressor() # again default parameters for now but might change them soon
res_rf = run_model(rf)

print(res_rf)

{'Train Error': 679.8884269452279, 'Test Error': 1699.9979935825118}


In [ ]:
# Sixth (and Final) model: kNN Regressor

# will need to do two cross-validations: one for picking best 'k' and the other to get LOOCV error for knn regressor
# BUT first let's fit knn with default k=5
# knn = KNeighborsRegressor()
# knn.fit(X_ukr, y)

# loo = LeaveOneOut()

# mse_score_knn = cross_val_score(knn, X_ukr, y, cv=loo, scoring='neg_mean_squared_error', n_jobs=-1)
# mse_score_knn = [np.sqrt(abs(score)) for score in mse_score_knn]

# print(f'Root Mean squared error for Random Forest Regressor with LOOCV: {np.mean(mse_score_knn)}')



In [ ]:
# Finding the optimal k value for kNN regressor

# first let's find the optimal 'k'
kvals = list(range(3, 10))

loo = LeaveOneOut()
k_rmse = {} # this dict will contain the k value : corresponding rmse err
for k in kvals:
  knn_model = KNeighborsRegressor(n_neighbors=k)
  rmse = cross_val_score(knn_model, X_enc_ukr, y, cv=loo, scoring='neg_mean_squared_error', n_jobs=-1)
  rmse = [np.sqrt(abs(err)) for err in rmse]
  k_rmse[k] = np.mean(rmse)

print(k_rmse)

{3: 1179.1666666666665, 4: 1190.130357142857, 5: 1319.5271428571427, 6: 1372.0726190476191, 7: 1442.2877551020406, 8: 1443.0464285714286, 9: 1434.5611111111111}


In [ ]:
# K-nearest Neighbors Regression

knn = KNeighborsRegressor(n_neighbors=3)
res_knn = run_model(knn)
print(res_knn)



{'Train Error': 1424.6769450769032, 'Test Error': 2151.700137620109}
